<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=da6325e5ab8a681764097304474a7a367c5e2600501bf0d141f5e3c142cc0e93
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2


    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-08 10:44:28
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.40 C
-------------------
Today PnL: -89.25 K (-0.63%)
Current PnL: -27.90 L (-18.02%)
CY Booked + Current PnL: -13.61 L (-8.79%)
-------------------
Total profit:  1.43 L
Total loss:  -29.33 L
-------------------
Total Booked + Current PnL: 13.37 L (10.46%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.18%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 92.53 L (65.92%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 5.28%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.22,-16.80,20.30,0.09,17021.0,-16930.0,83847.0,94.66,37.0,M-SC,2.38,253.0,-0.99,0.62,10.16,OX40N,NTT,DURABLES
50,MASFIN,398.61,2.25,0.37,21.60,22.05,21241.0,360.0,98340.0,-13.28,62.0,H-SC,7.08,164.0,0.02,0.73,42.94,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-0.93,-13.05,15.03,0.02,22812.0,-22783.0,151776.0,131.58,50.0,M-SC,16.84,234.0,-1.00,1.12,32.02,OX40N,NTT,PAINTS
43,ITC,452.00,-0.25,-1.30,11.90,10.44,23506.0,-2606.0,197532.0,-41.15,47.0,X-LC,1.58,5.0,-0.11,1.46,4.19,X40,NTT,FMCG
35,ICICIGI,2252.93,-0.43,4.89,14.59,20.19,26676.0,8518.0,182838.0,-17.91,44.0,X-MC,6.17,68.0,0.32,1.35,20.74,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,398.61,2.25,0.37,21.60,22.05,21241.0,360.0,98340.0,-13.28,62.0,H-SC,7.08,164.0,0.02,0.73,42.94,XR,ATH,FINANCE
68,STARHEALTH,761.00,1.73,-9.87,63.39,47.26,160610.0,-27755.0,253368.0,23.11,38.0,H-SC,13.82,171.0,-0.17,1.87,35.75,XY24,NTT,INSURANCE
13,BSOFT,831.70,1.47,-20.39,89.41,50.79,100112.0,-28683.0,111970.0,2.05,74.0,H-SC,5.65,151.0,-0.29,0.83,30.37,XR,ATH,IT
56,RAJESHEXPO,518.00,1.02,-63.97,177.70,0.07,87837.0,-87747.0,49430.0,1711.87,50.0,L-SC,2.35,268.0,-1.00,0.37,22.94,OX40N,NTT,JEWELLERY
84,WIPRO,420.00,0.86,7.52,60.21,72.27,109699.0,12746.0,182194.0,-7.89,75.0,M-LC,2.79,99.0,0.12,1.35,14.16,XR,NTT,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.0,-3.42,-45.55,148.45,35.27,91778.0,-51725.0,61824.0,-57.75,22.0,L-SC,12.93,270.0,-0.56,0.46,51.32,XR,NTT,HOTELS
80,VALIANTORG,838.0,-3.42,-46.05,222.80,74.15,196652.0,-75341.0,88264.0,-259.08,39.0,H-SC,20.90,149.0,-0.38,0.65,13.14,XR,NTT,CHEMICALS
83,WHIRLPOOL,2270.0,-2.32,-25.67,144.74,81.92,123507.0,-29468.0,85330.0,-56.47,19.0,M-SC,8.15,236.0,-0.24,0.63,1.02,XR,NTT,DURABLES
60,REPCOHOME,880.0,-2.23,-14.24,121.33,89.82,298128.0,-40788.0,245717.0,-56.78,41.0,H-SC,7.58,160.0,-0.14,1.81,30.00,XY24,NTT,FINANCE
58,RECLTD,446.0,-2.05,-6.32,28.88,20.74,54967.0,-12842.0,190328.0,24.12,35.0,M-MC,6.46,189.0,-0.23,1.41,0.11,XY25,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,398.61,2.25,0.37,21.6,22.05,21241.0,360.0,98340.0,-13.28,62.0,H-SC,7.08,164.0,0.02,0.73,42.94,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.28,-7.91,125.77,107.91,171007.0,-11676.0,135968.0,-25.01,27.0,M-SC,11.57,216.0,-0.07,1.00,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.93,-13.05,15.03,0.02,22812.0,-22783.0,151776.0,131.58,50.0,M-SC,16.84,234.0,-1.00,1.12,32.02,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.22,-16.80,20.30,0.09,17021.0,-16930.0,83847.0,94.66,37.0,M-SC,2.38,253.0,-0.99,0.62,10.16,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.52,-24.27,49.83,13.47,101767.0,-65439.0,204228.0,-68.90,30.0,H-SC,2.84,158.0,-0.64,1.51,3.84,XY24,NTT,PAINTS
66,SIS,528.00,-1.23,-25.36,62.46,21.26,51764.0,-28157.0,82875.0,1953.37,43.0,H-SC,4.24,166.0,-0.54,0.61,11.97,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.86,7.52,60.21,72.27,109699.0,12746.0,182194.0,-7.89,75.0,M-LC,2.79,99.0,0.12,1.35,14.16,XR,NTT,IT
50,MASFIN,398.61,2.25,0.37,21.60,22.05,21241.0,360.0,98340.0,-13.28,62.0,H-SC,7.08,164.0,0.02,0.73,42.94,XR,ATH,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.86,7.52,60.21,72.27,109699.0,12746.0,182194.0,-7.89,75.0,M-LC,2.79,99.0,0.12,1.35,14.16,XR,NTT,IT
50,MASFIN,398.61,2.25,0.37,21.60,22.05,21241.0,360.0,98340.0,-13.28,62.0,H-SC,7.08,164.0,0.02,0.73,42.94,XR,ATH,FINANCE
38,INDIAMART,4810.62,-1.03,-1.66,110.21,106.72,133674.0,-2046.0,121290.0,-51.39,36.0,H-SC,1.34,139.0,-0.02,0.90,21.33,AR,ATH,MISC
37,IEX,219.00,-0.10,-1.48,50.86,48.63,100192.0,-2958.0,196996.0,-34.34,56.0,H-SC,14.09,136.0,-0.03,1.45,9.90,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,0.42,-1.36,37.50,35.63,77521.0,-2851.0,206723.0,-15.41,43.0,H-MC,3.15,119.0,-0.04,1.53,15.09,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-1.34,-22.89,92.53,48.45,143054.0,-45903.0,154603.0,-28.46,10.0,X-MC,10.62,64.0,-0.32,1.14,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-1.15,-38.35,119.29,35.20,94634.0,-49339.0,79331.0,3.97,24.0,X-SC,15.01,92.0,-0.52,0.59,0.00,X40,NTT,FOOTWEAR
54,PGHH,17907.65,-0.64,-6.59,43.19,33.76,81020.0,-13230.0,187590.0,-34.12,27.0,X-MC,5.09,57.0,-0.16,1.39,0.00,X40,ATH,FMCG
22,DIXON,18931.72,0.37,-11.37,37.19,21.58,46190.0,-15939.0,124200.0,-55.32,28.0,X-MC,5.32,56.0,-0.35,0.92,9.96,X40N,ATH,IT
75,TMPV,600.00,-0.44,-16.58,70.43,42.18,162407.0,-45817.0,230593.0,-25.52,29.0,X-LC,4.48,3.0,-0.28,1.70,0.00,XY24,NTT,AUTO


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,-0.75,-20.59,93.30,53.50,223556.0,-62141.0,239610.0,-61.59,35.0,X-MC,1.36,58.0,-0.28,1.77,6.43,XY24,NTT,FMCG
43,ITC,452.0,-0.25,-1.30,11.90,10.44,23506.0,-2606.0,197532.0,-41.15,47.0,X-LC,1.58,5.0,-0.11,1.46,4.19,X40,NTT,FMCG
3,ACC,3906.0,0.48,-24.06,116.17,64.15,209908.0,-57261.0,180690.0,-55.44,40.0,X-SC,1.66,82.0,-0.27,1.33,1.36,XY24,BTT,CEMENT
15,CAMS,4762.0,-1.95,-80.07,526.08,24.79,264092.0,-201656.0,50200.0,-81.09,43.0,X-SC,2.14,86.0,-0.76,0.37,22.72,X40N,NTT,MISC
36,ICICIPRULI,790.0,0.84,4.66,25.14,30.97,46502.0,8233.0,184971.0,-20.00,61.0,X-MC,2.23,75.0,0.18,1.37,17.80,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-1.15,-38.35,119.29,35.20,94634.0,-49339.0,79331.0,3.97,24.0,X-SC,15.01,92.0,-0.52,0.59,0.0,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-1.34,-22.89,92.53,48.45,143054.0,-45903.0,154603.0,-28.46,10.0,X-MC,10.62,64.0,-0.32,1.14,0.0,X40N,ATH,FINANCE
55,QUESS,424.00,-1.32,-32.90,112.89,42.84,49238.0,-21390.0,43616.0,-55.92,32.0,X-SC,37.32,83.0,-0.43,0.32,0.0,XY24,NTT,MISC
53,PAGEIND,51605.08,-0.88,-9.31,39.00,26.07,57915.0,-15240.0,148500.0,-33.27,30.0,X-MC,11.19,55.0,-0.26,1.10,0.0,X40,ATH,APPARELS
54,PGHH,17907.65,-0.64,-6.59,43.19,33.76,81020.0,-13230.0,187590.0,-34.12,27.0,X-MC,5.09,57.0,-0.16,1.39,0.0,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.0,-1.32,-32.90,112.89,42.84,49238.0,-21390.0,43616.0,-55.92,32.0,X-SC,37.32,83.0,-0.43,0.32,0.00,XY24,NTT,MISC
15,CAMS,4762.0,-1.95,-80.07,526.08,24.79,264092.0,-201656.0,50200.0,-81.09,43.0,X-SC,2.14,86.0,-0.76,0.37,22.72,X40N,NTT,MISC
59,RELAXO,1176.0,-0.56,-47.90,192.36,52.31,145468.0,-69537.0,75623.0,-44.54,34.0,X-SC,5.72,91.0,-0.48,0.56,1.23,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.0,-1.15,-38.35,119.29,35.20,94634.0,-49339.0,79331.0,3.97,24.0,X-SC,15.01,92.0,-0.52,0.59,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,1486.0,-0.09,-2.93,27.83,24.08,34294.0,-3725.0,123225.0,-10.78,33.0,X-SC,5.86,89.0,-0.11,0.91,16.11,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.28,-12.01,35.57,19.28,108272.0,-41565.0,304391.0,-23.30,70.0,X-LC,6.32,1.0,-0.38,2.25,12.53,X40,ATH,IT
41,INFY,2275.00,0.54,10.62,40.00,54.87,138450.0,33237.0,346125.0,-13.85,72.0,X-LC,3.41,2.0,0.24,2.56,19.78,X40,BTT,IT
75,TMPV,600.00,-0.44,-16.58,70.43,42.18,162407.0,-45817.0,230593.0,-25.52,29.0,X-LC,4.48,3.0,-0.28,1.70,0.00,XY24,NTT,AUTO
81,VBL,671.64,-1.53,-4.53,42.12,35.67,127000.0,-14323.0,301519.0,-15.92,54.0,X-LC,4.91,4.0,-0.11,2.23,8.69,X40N,ATH,FMCG
43,ITC,452.00,-0.25,-1.30,11.90,10.44,23506.0,-2606.0,197532.0,-41.15,47.0,X-LC,1.58,5.0,-0.11,1.46,4.19,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-1.35,-36.63,109.70,32.89,53672.0,-28280.0,48926.0,-703.98,60.0,L-MC,6.43,259.0,-0.53,0.36,34.74,XR,NTT,BANKS
6,ASIANTILES,137.00,-1.10,-18.13,121.90,81.67,93625.0,-17005.0,76805.0,6900.00,35.0,L-SC,15.56,271.0,-0.18,0.57,48.74,XR,NTT,CERAMICS
50,MASFIN,398.61,2.25,0.37,21.60,22.05,21241.0,360.0,98340.0,-13.28,62.0,H-SC,7.08,164.0,0.02,0.73,42.94,XR,ATH,FINANCE
13,BSOFT,831.70,1.47,-20.39,89.41,50.79,100112.0,-28683.0,111970.0,2.05,74.0,H-SC,5.65,151.0,-0.29,0.83,30.37,XR,ATH,IT
39,INDIGOPNTS,1408.00,-0.93,-13.05,15.03,0.02,22812.0,-22783.0,151776.0,131.58,50.0,M-SC,16.84,234.0,-1.00,1.12,32.02,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,1.47,-20.39,89.41,50.79,100112.0,-28683.0,111970.0,2.05,74.0,H-SC,5.65,151.0,-0.29,0.83,30.37,XR,ATH,IT
5,ASIANPAINT,3460.26,-0.61,3.12,17.28,20.95,44865.0,7867.0,259635.0,2.04,75.0,X-LC,18.91,31.0,0.18,1.92,39.87,X40,ATH,PAINTS
32,HCLTECH,1922.01,0.49,9.76,13.64,24.73,36219.0,23610.0,265534.0,14.46,72.0,X-LC,5.55,7.0,0.65,1.96,26.48,X40,ATH,IT
84,WIPRO,420.00,0.86,7.52,60.21,72.27,109699.0,12746.0,182194.0,-7.89,75.0,M-LC,2.79,99.0,0.12,1.35,14.16,XR,NTT,IT
65,SIEMENS,4671.50,-0.97,-11.50,41.83,25.51,68888.0,-21410.0,164685.0,1.14,57.0,H-LC,3.84,49.0,-0.31,1.22,21.36,AR,ATH,ELECTRICAL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.87
1,25,45.82
2,50,76.71


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.96
MC    30.21
LC    24.89
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.31
X40      22.52
X40N     10.86
XR       10.06
AR        9.42
XY25      9.37
OX40N     7.74
SR        1.01
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.51
X-MC    23.47
X-LC    21.29
M-SC    12.12
X-SC     6.93
H-MC     4.97
M-MC     1.41
L-SC     1.40
M-LC     1.35
H-LC     1.22
L-LC     1.03
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.38,-6.95,41.53
IT,12.93,-15.19,77.47
FINANCE,9.58,-19.55,70.06
ELECTRICAL,6.26,-10.73,51.28
PAINTS,6.18,-10.42,26.82
MISC,6.14,-56.28,120.31
INSURANCE,4.59,-1.77,37.64
PHARMA,4.08,-1.26,33.67
AUTO,3.46,-22.68,75.28


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3201987.0,21
XR,1301275.0,13
AR,1298288.0,10
X40,992393.0,14
X40N,978685.0,9
OX40N,743985.0,10
XY25,374200.0,6
SR,283142.0,2
MH,79295.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3658754.0,25
M-SC,1418663.0,15
X-MC,1307956.0,16
X-SC,980882.0,8
X-LC,880464.0,11
H-MC,406072.0,3
L-SC,273240.0,3
M-LC,109699.0,1
H-LC,68888.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1284928.0      6
           AR         891782.0      5
M-SC       XY24       804641.0      6
H-SC       XR         778136.0      7
X-SC       X40N       503485.0      3
X-MC       X40        457124.0      7
           XY24       389102.0      3
X-LC       X40        351312.0      5
H-SC       OX40N      341471.0      4
M-SC       OX40N      314677.0      5
X-SC       XY24       293440.0      3
H-SC       SR         283142.0      2
X-MC       X40N       274125.0      4
X-LC       XY24       236442.0      2
H-MC       AR         212638.0      2
X-LC       X40N       201075.0      2
H-MC       XY24       193434.0      1
X-MC       XY25       187605.0      2
L-SC       XR         185403.0      2
X-SC       X40        183957.0      2
M-SC       XR         174365.0      2
           AR         124980.0      2
M-LC       XR         109699.0      1
X-LC       XY25        91635.0      2
L-SC       OX40N       87837.0      1
H-SC       MH          79295.0      1
H-LC       AR          68888.0      1
M-MC       XY25        54967.0      1
L-MC       XR          53672.0      1
L-LC       XY25        39993.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 38.0 seconds
